In [1]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 70.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 46.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 105.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 115.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/894.0 kB ? eta -:--:--
   ----------

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "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"
os.environ['LA']

In [37]:
os.environ['OPENAI_API_KEY'] = "sk-a3cde32217314715adb67f38f2d23723"


In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key="sk-a3cde32217314715adb67f38f2d23723",    # Add your DashScope API key here
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

# # Test it with a simple completion
# response = client.chat.completions.create(
#     model="qwen-max",    # Start with this model first
#     messages=[
#         {"role": "user", "content": "Write a one-sentence bedtime story about a unicorn."}
#     ]
# )

# print(response.choices[0].message.content)


Under the soft glow of the moon, a gentle unicorn named Luna pranced through the whispering forest, her silvery mane shimmering as she sprinkled stardust over sleeping flowers.


In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import DashScopeEmbeddings  # Changed this

# Configure API key
dashscope_api_key = "sk-a3cde32217314715adb67f38f2d23723"
dashscope_base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1"

#### INDEXING ####

# Load Documents
# loader = WebBaseLoader(
#     web_paths=("https://shamela.ws/book/1655/1#p1",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             "span", class_=("c1", "c2",'c3','c4','c5')
#     ),
# ))
# docs = loader.load()
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Use DashScope's embedding model
embeddings = DashScopeEmbeddings(
    dashscope_api_key=dashscope_api_key,
    model="text-embedding-v1"  # DashScope's embedding model
)

# Create vectorstore
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=embeddings
)

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# Configure LLM for DashScope
llm = ChatOpenAI(
    model_name="qwen-max",  # or whatever model you have access to
    openai_api_key=dashscope_api_key,
    openai_api_base=dashscope_base_url,
    temperature=0
)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
response = rag_chain.invoke("ما هى أحكام الوضوء ؟")
print(response)

BadRequestError: Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Input data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-8f1352e2-a119-9f61-8526-2dfe5dc20c4a', 'request_id': '8f1352e2-a119-9f61-8526-2dfe5dc20c4a'}

In [94]:
response = rag_chain.invoke("متى يُنَجَّسُ الماء ؟")
print(response)

الماء لا ينجس إذا كان في كمية كبيرة، بحيث لا يتغير لونه أو طعمه أو رائحته بسبب النجاسة. وفقًا للسياق المعطى، إذا كان الماء قُلَّتين (أي كمية كبيرة نسبياً)، فإنه لا يحمل النجاسة.


In [85]:
# Load Documents
l = [f"https://shamela.ws/book/1655/{i}#p1" for i in range(1,31)]
l= tuple(l)

loader = WebBaseLoader(
    web_paths=l,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "span", class_=("c1", "c2",'c3','c4','c5')
    ),
))
docs = loader.load()
print("Number of loaded documents:", len(docs)) # Debug line

# Check the content of the first document to see if anything was loaded
if docs:
    print("Content of the first document:", docs[1].page_content[:200])
else:
    print("No documents loaded. Check your bs_kwargs and URL.")

KeyboardInterrupt: 

In [87]:
docs[2].page_content[:200]

'لَمْ يَكُنْ مَوْجُودًا فِيهِ فَإِنْ قَالَ قَائِلٌ: وَلَمْ يَتَغَيَّرْ وَاحِدٌ مِنْهُمَا قِيلَ: قَالَ:«إذَا كَانَ الْمَاءُ قُلَّتَيْنِ لَمْ يَحْمِلْ نَجَسًا أَوْ خَبَثَا»«إذَا كَانَ الْمَاءُ قُلَّتَيْن'